<a href="https://colab.research.google.com/github/princessivy/Analyzing_CSGO_SkinPrices/blob/main/CSGO_Significance_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [34]:
import pandas as pd
import numpy as np
from google.colab import drive
from pathlib import Path

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

#Daten einlesen

In [5]:
data = pd.read_csv(r'/content/drive/MyDrive/SA_BA/Data/allitems_cleaned_3.csv')

# Datenaufbereitung für Signifikanztest

In [6]:
# übrige Features:
data.columns

Index(['index', 'assetId', 'family', 'lock', 'version', 'salePrice',
       'category', 'subCategory', 'pattern', 'wear', 'exterior', 'quality',
       'rarity', 'collection', 'stickers', 'souvenir', 'stattrak',
       'colorfulness', 'cats_colorfulness'],
      dtype='object')

Die Features *index* und *assetId* können direkt gelöscht werden, da diese nur zur Unterstützung der Visualisierungen dienten.

In [7]:
data = data.drop(['index', 'assetId'], axis = 1)

Nun werden die kategorischen Features in nummerische umgewandelt. 

In [8]:
# kategorische Features filtern
data.dtypes

family                object
lock                   int64
version               object
salePrice            float64
category              object
subCategory           object
pattern                int64
wear                 float64
exterior              object
quality               object
rarity                object
collection            object
stickers               int64
souvenir               int64
stattrak               int64
colorfulness         float64
cats_colorfulness     object
dtype: object

Kategorisch:
* family
* version
* category
* subCategory
* exterior
* quality
* rarity
* collection
* cats_colorfulness

**Einordnung Skalenniveaus:**

Davon ordinalskaliert mit Anzahl Ausprägungen:
* exterior, 5
* rarity, 7
* cats_colorfulness, 6

Davon nominalskaliert mit Anzahl Ausprägungen:
* family, 911
* version, 20
* category, 6
* subCategory, 61
* quality, 5
* collection, 75

**Im Folgenden werden die ordinalskalierten Daten encodiert (Label-Encoding):**  
*(mir ist bewusst, dass es hierfür auch from sklearn.preprocessing import LabelEncoder gibt, allerdings kann ich durch manuelles Arbeiten sicher gehen, dass die Zahlen in der richtigen Reihenfolge vergeben werden)*

**exterior**  
Reihenfolge von gut zu schlecht: Factory New, Minimal Wear, Field-Tested, Well-Worn, Battle-Scarred.  
Entsprechend enthält Battle-Scarred den niedrigsten und Factory New den höchsten Wert.

In [9]:
data.exterior.unique()

array(['Factory New', 'Minimal Wear', 'Field-Tested', 'Battle-Scarred',
       'Well-Worn'], dtype=object)

In [10]:
def encode_exterior(x):
  if x == 'Factory New':
    return 5
  elif x == 'Minimal Wear':
    return 4
  elif x == 'Field-Tested':
    return 3
  elif x == 'Well-Worn':
    return 2
  elif x == 'Battle-Scarred':
    return 1

In [11]:
data.exterior = data.exterior.apply(encode_exterior)

**rarity**  
Reihenfolge von gut zu schlecht: Extraordinary, Covert, Classified, Restricted, Mil-Spec Grade, Industrial Grade, Consumer Grade.  

In [12]:
data.rarity.unique()

array(['Covert', 'Restricted', 'Mil-Spec Grade', 'Classified',
       'Industrial Grade', 'Consumer Grade', 'Extraordinary'],
      dtype=object)

Bemerkung: In *Extraordinary* stecken alle Skins der Kategorie Gloves. Diese sind mit *Covert* gleichgestellt. Entsprechend werden diese in die gleichie Kategorie fallen.

In [13]:
data[data['rarity'] == 'Extraordinary'].category.unique()

array(['Gloves'], dtype=object)

In [14]:
def encode_rarity(x):
  if (x == 'Extraordinary') or (x == 'Covert'):
    return 6
  elif x == 'Classified':
    return 5
  elif x == 'Restricted':
    return 4
  elif x == 'Mil-Spec Grade':
    return 3
  elif x == 'Industrial Grade':
    return 2
  elif x == 'Consumer Grade':
    return 1

In [15]:
data.rarity = data.rarity.apply(encode_rarity)

**cats_colorfulness**

Reihenfolge von gut zu schlecht: extremely colorful, highly colorful, quite colorful, averagely colorful, moderately colorful, slightly colorful, not colorful.

In [16]:
data.cats_colorfulness.unique()

array(['highly colorful', 'quite colorful', 'slightly colorful',
       'moderately colorful', 'averagely colorful', 'extemely colorful'],
      dtype=object)

In [17]:
def encode_colors(x):
  if x == 'extemely colorful':
    return 6
  elif x == 'highly colorful':
    return 5
  elif x == 'quite colorful':
    return 4
  elif x == 'averagely colorful':
    return 3
  elif x == 'moderately colorful':
    return 2
  elif x == 'slightly colorful':
    return 1
  elif x == 'not colorful': #gibt's nicht
    return 0

In [18]:
data.cats_colorfulness = data.cats_colorfulness.apply(encode_colors)

Speicherpunkt 4:

In [ ]:
data.to_csv(r'allitems_cleaned_4.csv', index=False, header=True)

In [35]:
data = pd.read_csv(r'/content/drive/MyDrive/SA_BA/Data/allitems_cleaned_4.csv')

**Im folgenden werden die nominalskalierten Daten encodiert:**

* family, 911
* version, 20
* category, 6 - OHE?
* subCategory, 61
* quality, 5 - OHE?
* collection, 75
* pattern..?

##Beispiel Feature Category!

In [24]:
data['salePricelog'] = np.log(data['salePrice'])

In [25]:
data2 = pd.get_dummies(data.category, prefix='cat')

In [26]:
data3 = pd.concat([data, data2], axis=1)

In [27]:
y = data3['salePricelog']
X = data3[['rarity', 'wear', 'lock', 'cat_Gloves', 'cat_Heavy', 'cat_Knife', 'cat_Pistol', 'cat_Rifle', 'cat_SMG']]

In [30]:
model_lr = LinearRegression()

In [31]:
X = sm.add_constant(X)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [32]:
model = sm.OLS(y,X)
results = model.fit()

In [33]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           salePricelog   R-squared:                       0.770
Model:                            OLS   Adj. R-squared:                  0.770
Method:                 Least Squares   F-statistic:                 5.564e+04
Date:                Wed, 04 May 2022   Prob (F-statistic):               0.00
Time:                        16:12:02   Log-Likelihood:            -2.3590e+05
No. Observations:              132807   AIC:                         4.718e+05
Df Residuals:                  132798   BIC:                         4.719e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.6416      0.012   -217.998      0.0